In [4]:
!pip install mpl_toolkits


ERROR: Could not find a version that satisfies the requirement mpl_toolkits (from versions: none)
ERROR: No matching distribution found for mpl_toolkits


In [3]:
import tensorflow

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV

def analyzePCA(X, pca, explained_threshold=0.95):
    """Plot cumulative explaned variance of PCA"""
    pca.fit(X)
    pca_variance = pca.explained_variance_ratio_
    plt.figure(figsize=(15,7))
    plt.plot(np.cumsum(pca_variance))
    plt.hlines(explained_threshold, 0, len(pca_variance))
    plt.title("Number of components to explain {}% of variance: {}".format(
        100*explained_threshold, np.argwhere(np.cumsum(pca_variance)>0.95)[0]
    ))
    plt.xlabel('number of components')
    plt.ylabel('cumulative explained variance');

# feats_lst = []
# for i in feats_woe:
#     if i.find('cross_') != 0:
#         feats_lst.append(i)
        
scaler = StandardScaler()

df_scaled = scaler.fit_transform(df.drop(['DDateRequest', 'demand_id1', 'RequestIDRR', 'NPL30'], axis = 1))

pca = PCA()
analyzePCA(df_scaled, pca)

In [ ]:
%%time

from tune_sklearn import TuneSearchCV
from sklearn import datasets
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import pandas as pd

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3)

# A parameter grid for XGBoost
# params = {
#     "min_child_weight": list(range(2, 25)),
#     "gamma": list(np.arange(0.1, 5, 0.1)),
#     "subsample": list(np.arange(0.1, 0.9, 0.025)),
#     "colsample_bytree": list(np.arange(0.1, 0.9, 0.05)),
#     "max_depth": list(range(2, 15)),
#     "n_estimators": list(range(3, 250)),
#     "learning_rate": [0.0001, 0.001, 0.01, 0.1, 0.15, 0.2, 0.25, 0.3]
# }

params = {
    'n_estimators'     : list(range(2, 3000)),  # noqa: E221,E241
    'max_depth'        : list(range(2, 4)),   # noqa: E221,E241
    'gamma'            : list(np.arange(0.1, 5, 0.1)),  # noqa: E221,E241
    'eta'              : list(np.arange(0.001, 0.5, 0.001)),  # noqa: E221,E241
    'colsample_bytree' : list(np.arange(0.005, 0.995, 0.005)),    # noqa: E221,E241
    'subsample'        : list(np.arange(0.005, 0.995, 0.005)),    # noqa: E221,E241
    'reg_lambda'       : list(np.arange(0.1, .95, 0.05)),  # noqa: E221,E241
    'min_child_weight' : list(range(2, 125)),   # noqa: E221,E241
    'max_leaves'       : list(range(2, 125)),  # noqa: E221,E241
    "learning_rate": [0.0001, 0.001, 0.01, 0.1, 0.15, 0.16, 0.17, 0.18, 0.19]
}
xgb = XGBClassifier(
    learning_rate=0.02,
    n_estimators=50,
    objective="binary:logistic",
    nthread=5,
    # tree_method="gpu_hist"  # this enables GPU.
    # See https://github.com/dmlc/xgboost/issues/2819
)

digit_search = TuneSearchCV(
    xgb,
    param_distributions=params,
    n_trials=3,
    # use_gpu=True # Commented out for testing on github actions,
    # but this is how you would use gpu
)

digit_search.fit( df[df['oot_flg'] == 0][cols], df[df['oot_flg'] == 0][target])


In [ ]:
Дан список названий колонок column_list и датасет df, 
в котором колонки частично совпадают со списком. Напиши код, 
который создаст новый датасет df1 с колонками из списка column_list, 
перенесет в новый датасет df1 все колонки, которые есть в списке из 
исходного df, и те, которых нет, заполнит нулями

In [1]:
import pandas as pd

# Пример данных
column_list = ['A', 'B', 'C', 'D', 'E']
data = {
    'A': [1, 2, 3],
    'C': [4, 5, 6],
    'E': [7, 8, 9]
}
df = pd.DataFrame(data)

# Создание нового датасета df1 с колонками из column_list и заполнением нулями
df1 = pd.DataFrame(columns=column_list)  # Создание датасета с заданными колонками

for column in df.columns:
    if column in column_list:
        df1[column] = df[column]  # Перенос совпадающих колонок
    else:
        df1[column] = 0  # Заполнение отсутствующих колонок нулями

# Вывод нового датасета df1
print(df1)

   A    B  C    D  E
0  1  NaN  4  NaN  7
1  2  NaN  5  NaN  8
2  3  NaN  6  NaN  9


In [ ]:
# Создание словаря с уникальными списками товаров в одном кластере
product_clusters = {}
for _, row in cluster_groups.iterrows():
    cluster_id = row['cluster']
    product_ids = row['ID товара']
    for product_id in product_ids:
        if product_id not in product_clusters:
            product_clusters[product_id] = []

        # Добавление уникальных товаров в список без дубликатов
        for pid in product_ids:
            if pid != product_id and pid not in product_clusters[product_id]:
                product_clusters[product_id].append(pid)

In [ ]:
### Как работает эта часть кода:

1. **product_clusters**:
   - Создается пустой словарь product_clusters, который будет содержать соответствие ID товара и списков товаров в одном кластере.

2. **Итерация по группам кластеров**:
   - Цикл for _, row in cluster_groups.iterrows() проходит по каждой строке в группах кластеров. Здесь row представляет собой строку из DataFrame.

3. **Извлечение ID кластера и товаров**:
   - Извлекается ID кластера (cluster_id) и список ID товаров (product_ids) из текущей строки.

4. **Формирование соответствий в словаре**:
   - Для каждого товара в списке product_ids проверяется, есть ли он уже в словаре product_clusters. Если нет, создается пустой список для этого товара.
   - Затем добавляются ID всех товаров из списка product_ids, за исключением самого товара, путем расширения списка текущим списком, используя список включений (list comprehension).

Например, если в кластере с ID 1 есть товары с ID [101, 102, 103], то после выполнения этой части кода словарь product_clusters будет иметь вид:
{
    101: [102, 103],
    102: [101, 103],
    103: [101, 102]
}


Таким образом, этот участок кода формирует соответствие ID товара и списков товаров из одного и того же кластера в словаре product_clusters.

In [ ]:
Чтобы избежать дубликатов в списках товаров в словаре product_clusters, 
можно модифицировать код таким образом, чтобы при добавлении товаров в 
список выполнялась проверка на их уникальность. Вот как можно отредактировать эту часть кода:

In [ ]:
### Изменения в коде:

1. **Дополнительная проверка для уникальности**:
   - Добавлена внутренняя проверка при добавлении товаров в 
список в рамках одного кластера. Перед добавлением товара в список проверяется, не содержится ли он там уже.

Теперь код будет добавлять только уникальные товары в список для каждого товара в словаре.
Это поможет избежать дубликатов и создать структуру, в которой каждый товар из одного кластера не повторяется в своем же списке. 

In [ ]:
import pandas as pd

# Загрузка данных из датасета
# Предполагается, что данные уже загружены в DataFrame df
# Пример: df = pd.read_csv("file.csv")

# Группировка данных по кластерам
cluster_groups = df.groupby('cluster')['ID товара'].apply(list).reset_index()

 Создание словаря с уникальными списками товаров в одном кластере
product_clusters = {}
for _, row in cluster_groups.iterrows():
    cluster_id = row['cluster']
    product_ids = row['ID товара']
    for product_id in product_ids:
        if product_id not in product_clusters:
            product_clusters[product_id] = []

        # Добавление уникальных товаров в список без дубликатов
        for pid in product_ids:
            if pid != product_id and pid not in product_clusters[product_id]:
                product_clusters[product_id].append(pid)

# Преобразование в JSON
import json
json_data = json.dumps(product_clusters, indent=4)

# Запись JSON в файл
with open('product_clusters.json', 'w') as json_file:
    json_file.write(json_data)


In [ ]:
 Имеется датасет pandas , 
    первая колонка - ID товара, со второй по пятую - 
    значения категорий товаров с первой по четвертую степень вложенности. 
    в шестой  - номер кластера. Нужно получить файл json, в котором каждому
    id товара будет соответствовать список из всех ID товара, которые находятся
    с ним в одних и тех же кластерах,  чтобы внутри каждого списка товаров в словаре не было дубликатов. 
    Каждый ID товара может входить в несколько разных кластеров.
    Дальше нужно проверить каждый ключ словаря на вхождение товара в группу товаров из списка  по соответствию категории уровня 2)
    Каждый список нужно пропустить через фильтр на соответствие категории товаров списку соответствующему разметке, в которой ключ
    - категория товара из ключа, значения - разные по уровню иерархии категории ( значения из одной из колонок 2-5) исходного датасета
    Напиши скрипт на Python для реализации этой задачи и поясни все части кода
   
    
    
    

In [ ]:
import pandas as pd
import json

# Создание тестового датасета
data = {
    'ID товара': [1, 2, 3],
    'Кат1': ['A', 'B', 'C'],
    'Кат2': ['M', 'N', 'O'],
    'Кат3': ['X', 'Y', 'Z'],
    'Кат4': ['P', 'Q', 'R'],
    'Кластер': [1, 2, 1]
}
df = pd.DataFrame(data)

# Шаг 1: Преобразование датасета в словарь
cluster_dict = df.groupby('Кластер')['ID товара'].apply(list).to_dict()

# Шаг 2: Создание списка соответствий категорий товаров
category_mapping = {
    'A': ['M', 'N', 'X'],
    'B': ['M', 'N', 'Y'],
    'C': ['M', 'O', 'Z']
}

# Шаг 3: Фильтрация товаров по категориям
result = {}
for cluster, products in cluster_dict.items():
    filtered_products = []
    for product in products:
        categories = df.loc[df['ID товара'] == product, 'Кат1':'Кат4'].values.flatten().tolist()
        if all(category_mapping.get(categories[1]) == categories[2:]):
            filtered_products.append(product)
    result[cluster] = filtered_products

# Шаг 4: Генерация JSON файла
with open('result.json', 'w') as file:
    json.dump(result, file, ensure_ascii=False, indent=4)
    